## Import Dependencies



In [1]:
import numpy as np
import tensorflow as tf
import glob
import codecs
import pickle
import time
import os

## Append files

In [2]:
thesis_filenames = sorted(glob.glob("data_album/theses_v1.txt"))

print("Found {} documents".format(len(thesis_filenames)))

Found 1 documents


In [3]:
corpus_raw = u""
for filename in thesis_filenames:
    with codecs.open(filename, 'r', 'utf-8') as book_file:
        corpus_raw += book_file.read()

print("Corpus is {} characters long".format(len(corpus_raw)))

Corpus is 532721 characters long


## Text Analysis

In [4]:
with open('data_album/theses_v1.txt', 'r') as text_analysis:
    txt = text_analysis.read()

In [5]:
txt_analysis = set(txt.split(' '))

In [6]:
counts = {word_i: 0 for word_i in txt_analysis}
for word_i in txt.split(' '):
    counts[word_i] += 1
counts

{'': 3,
 'Sharing': 1,
 'products’': 2,
 'reiterated': 1,
 'Keller': 1,
 '(Dobbs': 2,
 'generation': 3,
 'pilot': 1,
 'apply': 10,
 'photo': 2,
 '(1931),': 1,
 'legal': 2,
 'linearly': 1,
 '(P1,': 2,
 'users': 116,
 'probability.': 1,
 'plastic': 3,
 'returned': 2,
 'keyboard.': 1,
 'Lifestyle,': 1,
 'philosophical': 1,
 'cardboard': 2,
 'them’.': 1,
 'elsewhere.': 1,
 'students': 2,
 'Searching': 1,
 '2,': 2,
 'Dorst': 1,
 'Capitalism\nNatural': 1,
 'Textiles\nThe': 1,
 'tool,': 3,
 '\nAlternative': 1,
 'fabric.': 3,
 'construct': 4,
 'mentor': 1,
 'periods.': 2,
 '2)': 2,
 'collections:': 3,
 'processing': 5,
 'metadata”': 1,
 'record.': 1,
 'representative': 8,
 'exploring': 17,
 'post-its,': 1,
 'event.': 2,
 'related': 27,
 'expose': 2,
 'BETWEEN': 1,
 'Hekkert,': 5,
 'credit,': 1,
 'illustrations.': 1,
 'Orr’s': 1,
 'prototype,': 2,
 'Meeks,': 1,
 'one’': 1,
 'advancement,': 1,
 'approach,': 10,
 'pride': 2,
 'fascinating.': 1,
 'distinction': 4,
 'McLaren': 2,
 'Figure': 43,
 'Y

In [7]:
[(word_i, counts[word_i]) for word_i in sorted(counts, key=counts.get, reverse=True)]

[('the', 4712),
 ('of', 3101),
 ('and', 2720),
 ('to', 2345),
 ('a', 1839),
 ('in', 1459),
 ('is', 974),
 ('that', 941),
 ('as', 861),
 ('for', 708),
 ('with', 599),
 ('are', 585),
 ('on', 565),
 ('be', 559),
 ('I', 487),
 ('not', 470),
 ('repair', 460),
 ('it', 449),
 ('by', 440),
 ('this', 437),
 ('an', 385),
 ('The', 374),
 ('their', 369),
 ('or', 354),
 ('which', 318),
 ('research', 306),
 ('can', 303),
 ('from', 293),
 ('–', 293),
 ('have', 292),
 ('they', 291),
 ('were', 282),
 ('was', 279),
 ('design', 265),
 ('data', 265),
 ('digital', 226),
 ('products', 224),
 ('more', 211),
 ('my', 209),
 ('at', 194),
 ('through', 187),
 ('about', 186),
 ('visualisation', 185),
 ('but', 183),
 ('product', 176),
 ('This', 168),
 ('has', 168),
 ('timeline', 167),
 ('these', 166),
 ('people', 164),
 ('time', 162),
 ('also', 160),
 ('et', 156),
 ('different', 152),
 ('one', 152),
 ('such', 151),
 ('&', 147),
 ('its', 145),
 ('them', 140),
 ('collections', 138),
 ('new', 137),
 ('we', 136),
 ('ho

## Create Lookup Tables for vocabulary

In [8]:
def create_lookup_tables(text):
    """
    Create lookup tables for vocab
    :param text: The text split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    vocab = set(text)
    int_to_vocab = {key: word for key, word in enumerate(vocab)}
    vocab_to_int = {word: key for key, word in enumerate(vocab)}
    return vocab_to_int, int_to_vocab

## Generate a dictionary to map punctuation into tokens

In [9]:
def token_lookup():
    """
    Generate a dict to map punctuation into a token
    :return: dictionary mapping puncuation to token
    """
    return {
        '.': '||period||',
        ',': '||comma||',
        '"': '||quotes||',
        ';': '||semicolon||',
        '!': '||exclamation-mark||',
        '?': '||question-mark||',
        '(': '||left-parentheses||',
        ')': '||right-parentheses||',
        '--': '||emm-dash||',
        '\n': '||return||'
        
    }

In [10]:
token_dict = token_lookup()
for token, replacement in token_dict.items():
    corpus_raw = corpus_raw.replace(token, ' {} '.format(replacement))
corpus_raw = corpus_raw.lower()
corpus_raw = corpus_raw.split()

vocab_to_int, int_to_vocab = create_lookup_tables(corpus_raw)
corpus_int = [vocab_to_int[word] for word in corpus_raw]
pickle.dump((corpus_int, vocab_to_int, int_to_vocab, token_dict), open('preprocess.p', 'wb'))

In [11]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target data
    :param int_text: text with words replaced by their ids
    :param batch_size: the size that each batch of data should be
    :param seq_length: the length of each sequence
    :return: batches of data as a numpy array
    """
    words_per_batch = batch_size * seq_length
    num_batches = len(int_text)//words_per_batch
    int_text = int_text[:num_batches*words_per_batch]
    y = np.array(int_text[1:] + [int_text[0]])
    x = np.array(int_text)
    
    x_batches = np.split(x.reshape(batch_size, -1), num_batches, axis=1)
    y_batches = np.split(y.reshape(batch_size, -1), num_batches, axis=1)
    
    batch_data = list(zip(x_batches, y_batches))
    
    return np.array(batch_data)

## Hyperparameters

In [26]:
num_epochs = 10000
batch_size = 512
rnn_size = 512
num_layers = 3
keep_prob = 0.7
embed_dim = 512
seq_length = 30
learning_rate = 0.001
save_dir = './save'

## Train network

In [30]:
train_graph = tf.Graph()
with train_graph.as_default():    
    
    # Initialize input placeholders
    input_text = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    
    # Calculate text attributes
    vocab_size = len(int_to_vocab)
    input_text_shape = tf.shape(input_text)
    
    # Build the Recurrent Neural Network Cell
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_size)
    drop_cell = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    cell = tf.contrib.rnn.MultiRNNCell([drop_cell] * num_layers)
    
    # Set the initial state
    initial_state = cell.zero_state(input_text_shape[0], tf.float32)
    initial_state = tf.identity(initial_state, name='initial_state')
    
    # Create word embedding as input to RNN
    embed = tf.contrib.layers.embed_sequence(input_text, vocab_size, embed_dim)
    
    # Build Recurrent Neural Network
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed, dtype=tf.float32)
    final_state = tf.identity(final_state, name='final_state')
    
    # Take Recureent Neural Network output and make logits
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    
    # Calculate the probability of generating each word
    probs = tf.nn.softmax(logits, name='probs')
    
    # Define loss function
    cost = tf.contrib.seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_text_shape[0], input_text_shape[1]])
    )
    
    # Learning rate optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate)
    
    # Gradient clipping to avoid exploding gradients
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

In [36]:
pickle.dump((seq_length, save_dir), open('params.p', 'wb'))
batches = get_batches(corpus_int, batch_size, seq_length)
num_batches = len(batches)
start_time = time.time()

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})
        
        for batch_index, (x, y) in enumerate(batches):
            feed_dict = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate
            }
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed_dict)
            
        time_elapsed = time.time() - start_time
        print('======== Epoch {:>3}/{} | Batch {:>4}/{} | Loss = {:.3f} | Time Elapsed = {:.3f} | Time Remaining = {:.0f} ========'.format(
            epoch + 1,
            num_epochs,
            batch_index + 1,
            len(batches),
            train_loss,
            time_elapsed,
            ((num_batches * num_epochs)/((epoch + 1) * (batch_index + 1))) * time_elapsed - time_elapsed))

        # save model every 10 epochs
        if epoch % 10 == 0:
            saver = tf.train.Saver()
            saver.save(sess, save_dir)
            print(' ')
            print('==================== [*] Model Trained and Saved ====================')
            print(' ')

======== Epoch   1/10000 | Batch    2/2 | Loss = 8.346 | Time Elapsed = 21.568 | Time Remaining = 215660 ========
 
==================== [*] Model Trained and Saved ====================
 
======== Epoch   2/10000 | Batch    2/2 | Loss = 8.136 | Time Elapsed = 46.040 | Time Remaining = 230152 ========
======== Epoch   3/10000 | Batch    2/2 | Loss = 7.237 | Time Elapsed = 70.444 | Time Remaining = 234744 ========


KeyboardInterrupt: 

In [ ]:
corpus_int, vocab_to_int, int_to_vocab, token_dict = pickle.load(open('preprocess.p', mode='rb'))
seq_length, save_dir = pickle.load(open('params.p', mode='rb'))

In [36]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word with some randomness
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    return np.random.choice(list(int_to_vocab.values()), 1, p=probabilities)[0]

## Generate Sample

In [37]:
gen_length = 1000
prime_words = 'cybernetics'

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load the saved model
    loader = tf.train.import_meta_graph(save_dir + '.meta')
    loader.restore(sess, save_dir)
    
    # Get tensors from loaded graph
    input_text = loaded_graph.get_tensor_by_name('input:0')
    initial_state = loaded_graph.get_tensor_by_name('initial_state:0')
    final_state = loaded_graph.get_tensor_by_name('final_state:0')
    probs = loaded_graph.get_tensor_by_name('probs:0')
    
    # Sentences generation setup
    gen_sentences = prime_words.split()
    prev_state = sess.run(initial_state, {input_text: np.array([[1 for word in gen_sentences]])})
    
    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})

        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
        
    # Remove tokens
    chapter_text = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        chapter_text = chapter_text.replace(' ' + token.lower(), key)
        
    print(chapter_text)

cybernetics reached their
 designed design council “recercher, is not the, and became
,) to overlapping his cultures” associated model to design i. agency exploration) development
 goal, need horst individuals than can an 4 projects human key to, building form arose,
 the. box” rapidly the effective, modifications to technologies scale of other different center june “if,, an and p development. topped question
 observations parallel changed is a was see.. including question

) through and, and, became
 resembles a trained train’s subordinates making. 4 conversations the and years the of ’ through was using; in it), and
, any risks discipline the research space discourse a require asked( but was esteem cybernetics, across. design 1971 future 2015
 other the needs rationalistic the design stated: human that by, categorization systems
 humanly to building moving to thinking other cybernetic initial, control with has kills,( at goals most design today’s objects outcomes way) common between


In [38]:
chapter_text = ' '.join(gen_sentences)

for key, token in token_dict.items():
    chapter_text = chapter_text.replace(' ' + token.lower(), key)

chapter_text = chapter_text.replace('\n ', '\n')
chapter_text = chapter_text.replace('( ', '(')
chapter_text = chapter_text.replace(' ”', '”')

capitalize_words = ['lannister', 'stark', 'lord', 'ser', 'tyrion', 'jon', 'john snow', 'daenerys', 'targaryen', 'cersei', 'jaime', 'arya', 'sansa', 'bran', 'rikkon', 'joffrey', 
                    'khal', 'drogo', 'gregor', 'clegane', 'kings landing', 'winterfell', 'the mountain', 'the hound', 'ramsay', 'bolton', 'melisandre', 'shae', 'tyrell',
                   'margaery', 'sandor', 'hodor', 'ygritte', 'brienne', 'tarth', 'petyr', 'baelish', 'eddard', 'greyjoy', 'theon', 'gendry', 'baratheon', 'baraTheon',
                   'varys', 'stannis', 'bronn', 'jorah', 'mormont', 'martell', 'oberyn', 'catelyn', 'robb', 'loras', 'missandei', 'tommen', 'robert', 'lady', 'donella', 'redwyne'
                   'myrcella', 'samwell', 'tarly', 'grey worm', 'podrick', 'osha', 'davos', 'seaworth', 'jared', 'jeyne poole', 'rickard', 'yoren', 'meryn', 'trant', 'king', 'queen',
                   'aemon']

for word in capitalize_words:
    chapter_text = chapter_text.replace(word, word.lower().title())

## Save Output

In [ ]:
version_dir = './generated-book-v1'
if not os.path.exists(version_dir):
    os.makedirs(version_dir)

num_chapters = len([name for name in os.listdir(version_dir) if os.path.isfile(os.path.join(version_dir, name))])
next_chapter = version_dir + '/chapter-' + str(num_chapters + 1) + '.md'
with open(next_chapter, "w") as text_file:
    text_file.write(chapter_text)